Copyright **`(c)`** 2024 Giovanni Squillero `<giovanni.squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free for personal or classroom use; see [`LICENSE.md`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.  

# Set Cover problem

See: https://en.wikipedia.org/wiki/Set_cover_problem

In [2]:
from random import random, seed
from itertools import product
import numpy as np

from icecream import ic
from tqdm.auto import tqdm

## Reproducible Initialization

If you want to get reproducible results, use `rng` (and restart the kernel); for non-reproducible ones, use `np.random`.

In [15]:
UNIVERSE_SIZE = 1000
NUM_SETS = 100
DENSITY = 0.2

rng = np.random.Generator(np.random.PCG64([UNIVERSE_SIZE, NUM_SETS, int(10_000* DENSITY)]))

In [16]:
# DON'T EDIT THESE LINES!

SETS = np.random.random((NUM_SETS, UNIVERSE_SIZE)) < DENSITY
for s in range(UNIVERSE_SIZE):
    if not np.any(SETS[:, s]):
        SETS[np.random.randint(NUM_SETS), s] = True
COSTS = np.power(SETS.sum(axis=1), 1.1)

## Helper Functions

In [17]:
def valid(solution):
    """Checks wether solution is valid (ie. covers all universe)"""
    return np.all(np.logical_or.reduce(SETS[solution]))


def cost(solution):
    """Returns the cost of a solution (to be minimized)"""
    return COSTS[solution].sum()


def fitness(solution: np.ndarray)-> float:
  return (valid(solution),-cost(solution))

## Algorithm 1: Random Mutation Hill Climbing
This is the first algorithm showed in class by the professor. Here I run it for the dimensions chosen to compare its performance to other algorithms. Instead of starting with `solution = rng.random(NUM_SETS) <  1`, I start with `solution = rng.random(NUM_SETS) <  .2`, and change the starting solution until I find a valid solution.

In [18]:
def tweak(solution: np.ndarray)-> np.ndarray:
  new_sol = solution.copy()
  index = np.random.randint(0, NUM_SETS)
  new_sol[index] = not new_sol[index]
  return new_sol

In [19]:
solution = rng.random(NUM_SETS) <  .2

while not valid(np.where(solution)[0]):
  solution = np.random.rand(NUM_SETS) < .2

solution_fitness = fitness(solution)
ic(fitness(solution))

for steps in range(10000):
  new_solution = tweak(solution)
  new_solution_fitness = fitness(new_solution)

  if new_solution_fitness > solution_fitness:
    solution = new_solution
    solution_fitness = new_solution_fitness

ic(fitness(solution))

ic| fitness(solution): (True, -9226.990267728956)
ic| fitness(solution): (True, -6460.043211505017)


(True, -6460.043211505017)

## Algorithm 2: RMHC with multiple mutations
Similar considerations to RMHC.

In [20]:
def multiple_mutation(solution: np.ndarray)-> np.ndarray:
  mask = rng.random(NUM_SETS) < .01
  new_sol = np.logical_xor(solution, mask)
  return new_sol

In [21]:
solution = rng.random(NUM_SETS) <  .2

while not valid(np.where(solution)[0]):
  solution = np.random.rand(NUM_SETS) < .2

solution_fitness = fitness(solution)
ic(fitness(solution))

for steps in range(10000):
  new_solution = multiple_mutation(solution)
  new_solution_fitness = fitness(new_solution)

  if new_solution_fitness > solution_fitness:
    solution = new_solution
    solution_fitness = new_solution_fitness

ic(fitness(solution))

ic| fitness(solution): (True, -8227.93445912554)
ic| fitness(solution): (True, -6125.526733861613)


(True, -6125.526733861613)

## Algorithm 3 : Simulated Annealing
Simulated annealing where as a tweak function I used the same from RMHC with multiple mutations.

In [22]:
def simulated_annealing(initial_temp=1000, cooling_rate=0.99, min_temp=1e-3, num_iterations=10000):

    current_solution = np.random.rand(NUM_SETS) < .5
    while not valid(np.where(current_solution)[0]):
        current_solution = np.random.rand(NUM_SETS) < .5

    current_fitness = fitness(current_solution)
    temp = initial_temp

    for _ in range(num_iterations):
        if temp < min_temp:
            break

        neighbor_solution = multiple_mutation(current_solution)

        if not valid(neighbor_solution):
            continue

        neighbor_fitness = fitness(neighbor_solution)
        delta_cost = neighbor_fitness[1] - current_fitness[1]

        if delta_cost > 0 or np.random.rand() < np.exp(delta_cost / temp):  # delta_cost is reversed due to -cost

            if neighbor_fitness > current_fitness:
                current_solution = neighbor_solution
                current_fitness = neighbor_fitness

        temp *= cooling_rate

    return np.where(current_solution)[0]

solution = simulated_annealing()
ic(fitness(solution))

ic| fitness(solution): (True, -6687.776633281468)


(True, -6687.776633281468)

## Algorithm 4 : Greedy Algorithm with preselection
Preselection:

*   Select all `SETS` with cost = 0
*   Select all `SETS` that are the only set covering a particular element

(This hint was given by the prof in class)

From the preselection the algorithm builds a solution with a greedy algorithm:

*   Iterates through all sets that are not yet in the solution.
*   Selects the set that covers the most uncovered elements for the least cost, based on the coverage-to-cost ratio (`coverage / COSTS[s])`
* Adds that set to the solution and repeats until all elements are covered

This algorithm has a better fitness than the other 3

In [23]:
def preselect_sets():
    selected = set()

    zero_cost_sets = np.where(COSTS == 0)[0]
    selected.update(zero_cost_sets)

    for element in range(UNIVERSE_SIZE):
        sets_covering_element = np.where(SETS[:, element])[0]
        if len(sets_covering_element) == 1:
            selected.add(sets_covering_element[0])

    preselected_solution = np.zeros(NUM_SETS, dtype=bool)  # Initialize as False (no sets selected)
    preselected_solution[list(selected)] = True  # Mark preselected sets as True
    return preselected_solution

def uncovered_elements(solution):
    """Return a boolean array indicating which elements are not yet covered by the current solution"""
    covered = np.logical_or.reduce(SETS[solution], axis=0)
    return np.where(~covered)[0]

def select_best_set(current_solution, uncovered):
    """Select the best set to add to the current solution to maximize coverage while minimizing cost"""
    best_set = None
    best_score = -np.inf

    for s in range(NUM_SETS):
        if current_solution[s]:
            continue

        coverage = np.sum(SETS[s, uncovered])
        if coverage == 0:
            continue

        score = coverage / COSTS[s]

        if score > best_score:
            best_set = s
            best_score = score

    return best_set


def build_solution():

    current_solution = preselect_sets()
    uncovered = uncovered_elements(current_solution)

    while len(uncovered) > 0:
        best_set = select_best_set(current_solution, uncovered)
        if best_set is None:
            raise ValueError("No set found to cover the remaining elements!")

        current_solution[best_set] = True
        uncovered = uncovered_elements(current_solution)

    return current_solution

solution = build_solution()

ic(fitness(solution))

ic| fitness(solution): (True, -5872.583350284479)


(True, -5872.583350284479)

## Chosen algorithm : Greedy Algorithm with preselection

Here the chosen algorithm is used for all the instances

In [24]:
def preselect_sets(NUM_SETS, SETS, COSTS):
    selected = set()

    zero_cost_sets = np.where(COSTS == 0)[0]
    selected.update(zero_cost_sets)

    for element in range(UNIVERSE_SIZE):
        sets_covering_element = np.where(SETS[:, element])[0]
        if len(sets_covering_element) == 1:
            selected.add(sets_covering_element[0])

    preselected_solution = np.zeros(NUM_SETS, dtype=bool)  # Initialize as False (no sets selected)
    preselected_solution[list(selected)] = True  # Mark preselected sets as True
    return preselected_solution

def uncovered_elements(solution, SETS):
    """Return a boolean array indicating which elements are not yet covered by the current solution"""
    covered = np.logical_or.reduce(SETS[solution], axis=0)
    return np.where(~covered)[0]

def select_best_set(current_solution, uncovered, NUM_SETS, SETS, COSTS):
    """Select the best set to add to the current solution to maximize coverage while minimizing cost"""
    best_set = None
    best_score = -np.inf

    for s in range(NUM_SETS):
        if current_solution[s]:
            continue

        coverage = np.sum(SETS[s, uncovered])
        if coverage == 0:
            continue

        score = coverage / COSTS[s]

        if score > best_score:
            best_set = s
            best_score = score

    return best_set


def build_solution(NUM_SETS, SETS, COSTS):

    current_solution = preselect_sets(NUM_SETS, SETS, COSTS)
    uncovered = uncovered_elements(current_solution, SETS)

    while len(uncovered) > 0:
        best_set = select_best_set(current_solution, uncovered, NUM_SETS, SETS, COSTS)
        if best_set is None:
            raise ValueError("No set found to cover the remaining elements!")

        current_solution[best_set] = True
        uncovered = uncovered_elements(current_solution, SETS)

    return current_solution

In [25]:
instances = [
    {"UNIVERSE_SIZE": 100, "NUM_SETS": 10, "DENSITY": 0.2},
    {"UNIVERSE_SIZE": 1_000, "NUM_SETS": 100, "DENSITY": 0.2},
    {"UNIVERSE_SIZE": 10_000, "NUM_SETS": 1_000, "DENSITY": 0.2},
    {"UNIVERSE_SIZE": 100_000, "NUM_SETS": 10_000, "DENSITY": 0.1},
    {"UNIVERSE_SIZE": 100_000, "NUM_SETS": 1_000, "DENSITY": 0.2},
    {"UNIVERSE_SIZE": 100_000, "NUM_SETS": 10_000, "DENSITY": 0.3},
]

for i, instance in enumerate(instances, start=1):

  UNIVERSE_SIZE = instance["UNIVERSE_SIZE"]
  NUM_SETS = instance["NUM_SETS"]
  DENSITY = instance["DENSITY"]

  rng = np.random.Generator(np.random.PCG64([UNIVERSE_SIZE, NUM_SETS, int(10_000* DENSITY)]))

  # DON'T EDIT THESE LINES!

  SETS = np.random.random((NUM_SETS, UNIVERSE_SIZE)) < DENSITY
  for s in range(UNIVERSE_SIZE):
      if not np.any(SETS[:, s]):
          SETS[np.random.randint(NUM_SETS), s] = True
  COSTS = np.power(SETS.sum(axis=1), 1.1)

  best_solution = build_solution(NUM_SETS, SETS, COSTS)

  print(f"Best fitness instance{i}: {fitness(best_solution)}")
  print("------------------")

Best fitness instance1: (True, -260.94571926897447)
------------------
Best fitness instance2: (True, -5556.8189611439675)
------------------
Best fitness instance3: (True, -97715.85450367707)
------------------
Best fitness instance4: (True, -1527629.1147761052)
------------------
Best fitness instance5: (True, -1772263.773201974)
------------------
Best fitness instance6: (True, -1760130.9299662877)
------------------
